# CX719

There 4 other CX's in this field: CX719, 879, 969 and 119. The brigtest one is CX719 and corresponds to the y values from 150 to the end. It is also the longest slit on this field. 

![](images/cx719.png)

![](images/cx719sexm.png)
- - - 


## Information

# Spectrum extraction

First import the require packages

In [1]:
from astropy.io import fits
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
from bokeh.layouts import  column, row
output_notebook()

Created directory /root/VIMOSReduced/cx0719/pyraf for cache


Loading BokehJS ...

## Defining the extraction parameters

First we define the parameters to extract the desire spectra:
    - Name of the extracted spectrum
    - Center, low and upper limits of the aperture
    - Background sample. Note that the background coordinates are relative to the aperture center and not image pixel coordinates so the endpoints need not be integer.
    - 2D spectra to use. Can use the SSEM or SEXM. The SSEM doesnt have the sky substracted, and the background substraction can be done with IRAF and apall. SEXM has the background substracted by the VIMOS pipeline so background can be set to none
    
To check what values can the website: https://vimos.manuelpm.me/

In [4]:
sourcename = 'cx719ssem'
databasegeneral = 'database/apcx719sexm'
filename = 'database/ap'+sourcename
fitsfilename = 'VI_SSEM_578511_2011-08-23T00:57:14.288_G475_MR_102307_Q1_hi.fits'

In [5]:
ape = fits.open(fitsfilename)
##For srfm[0].header["CTYPE1"] = 'LINEAR'
xn = ape[0].header["NAXIS1"]
refx = ape[0].header["CRVAL1"]
step = ape[0].header['CD1_1']
cr = ape[0].header['CRPIX1']
ape = ape[0].data
#
xlist = [ refx + step*(i - cr) for i in np.arange(ape.shape[1] +1) ]
#

#Default is half
y = ape[:,int(ape.shape[1]/2)]
x = list(range(y.shape[0]))
c2 = figure(x_axis_label='index',title='Wavelength',toolbar_location="above",active_scroll='wheel_zoom')
source2 = ColumnDataSource(data=dict(x=x,y=y))
sepaape = 5
sourceall2 = ColumnDataSource(data= dict([ (str(int(xlist[i])), ape[:,i]  ) for i in list(range(0,ape.shape[1],sepaape)) ]))

wavelist =  [ int(xlist[i]) for i in list(range(0,ape.shape[1],sepaape)) ]
sepawave = wavelist[1]- wavelist[0]


c2.line('x','y', source=source2)

callback2 = CustomJS(args=dict(source2 = source2, sourceall2 = sourceall2 ), code="""

        var data2 = source2.data;
        var data22 = sourceall2.data;
        var f = cb_obj.value;
        y = data2['y'];
        y2 = data22[f.toString()];


        for (i = 0; i < y.length; i++) {
            y[i] = y2[i];
        }
        source2.trigger('change');
    """)



slider2 = Slider(title="Wavelength", value=wavelist[int(len(wavelist)/2)], 
                 start=wavelist[0], end=wavelist[-1], step=sepawave,callback=callback2)


hover2 = HoverTool(
        tooltips=[
#            ("index", "$index"),
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
c2.add_tools(hover2)
c2.add_tools(tools.ResizeTool())

layout = column(slider2,c2)
show(layout)

## Aperture selection

Select the center the lower and upper limit, as well as the region for the background

In [25]:
center = 224
low = -2 
high = 2
b_low = [187- center,198-center]
b_up = [243-center,273-center]

## Exposure time

First we multiply the 2D spectra times the exposure time

In [26]:
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    
iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)

##  Define apertures and background sample in the database

We need to change the database of the aperture to define the center, lower and upper limit. Also to define the parameters of the background region to do the background substraction. This is the only way I have found to do it automatically. After this define the parameters in the .par file

In [27]:
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagenameor = lines
                imagename= lines.replace(lines.split()[1],sourcename)
        if 'center' in lines:
                numerocenteror = lines
                numerocenter = lines.replace(lines.split()[2], str(center))
        if 'low' in lines:
                numerolowor = lines
                numerolow = lines.replace(lines.split()[2],str(low))
        if 'high' in lines:
                numerohighor = lines
                numerohigh = lines.replace(lines.split()[2], str(high))
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],str(min(b_low)))
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],str(max(b_up)))
        if 'sample' in lines:
            sampleor = lines
            sampleb = '\t\tsample '+str(min(b_low))+':'+str(max(b_low))+','+str(min(b_up))+':'+str(max(b_up))+'\n'
            break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagenameor,imagename)
filedata = filedata.replace(numerocenteror,numerocenter)
filedata = filedata.replace(numerolowor,numerolow)
filedata = filedata.replace(numerohighor,numerohigh)
filedata = filedata.replace(bloworiginal,blow)
filedata = filedata.replace(buporiginal,bup)
filedata = filedata.replace(sampleor,sampleb)

                                
with open(filename,'w') as f:
    f.write(filedata)

In [28]:
%%bash -s "$filename"
cat $1

# Tue 02:43:50 14-Mar-2017
begin	aperture cx719sexm 1 1249. 217.5193
	image	cx719ssem
	aperture	1
	beam	1
	center	1249. 224
	low	-1248. -2
	high	1250. 2
	background
		xmin -37
		xmax 49
		function chebyshev
		order 1
		sample -37:-26,19:49
		naverage -3
		niterate 0
		low_reject 3.
		high_reject 3.
		grow 0.
	axis	2
	curve	5
		2.
		1.
		1.
		2499.
		0.



## Calling Apall

Now we define the parameters of apall, save it to the .par file and call the apall function.

In [29]:
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')
#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','average')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

## Plotting with Bokeh

Plot the text file after doing dispcor and exporting the spectra to a text file using the iraf routine wspectext. 

In [30]:
#Plotting
##For srfm[0].header["CTYPE1"] = 'LINEAR'
#Other way
#srfm = fits.open(sourcename+'.ms.fits')
#secondstar = srfm[0].data#[0][0]##[0][0] if using clean
#secondstar = srfm[0].data[0][0]
#xn = srfm[0].header["NAXIS1"]
#refx = srfm[0].header["CRVAL1"]
#step = srfm[0].header['CD1_1']
#cr = srfm[0].header['CRPIX1']
#
#xlist = [ refx + step*(i - cr) for i in np.arange(1, len(secondstar)+1) ]
#Create ColumnDataSource
#x = np.array(xlist)
#y = np.array(secondstar)

#Not sure if I need to do this
if os.path.exists(sourcename+'.dispcor.fits'):
    os.remove(sourcename+'.dispcor.fits')

iraf.dispcor(sourcename+'.ms.fits',sourcename+'.dispcor.fits')
iraf.wspectext(sourcename+'.dispcor.fits[*,1,1]',sourcename+'.txt',header='no')

x=[]
y=[]
with open(sourcename+'.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))


source = ColumnDataSource(data=dict(x=x,y=y))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra",
              active_scroll='wheel_zoom',plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

cx719ssem.ms.fits: Resampling using current coordinate system
cx719ssem.dispcor.fits: ap = 1, w1 =   3501.3, w2 =   9996.1, dw =      2.6, nw = 2499


## Normalizing the spectrum

We use the iraf routine continuum. We can define a sample to correctly normalize the desire range of wavelenght


In [31]:
sample = '5000:9000'
if os.path.exists(sourcename+'cont.fits'):
    os.remove(sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('input',sourcename+'.ms.fits')
iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('interactive','no')
iraf.noao.onedspec.continuum.setParam('sample',sample)
iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')

## Plotting the normalize spectra

We first define the desired range for the initial zoom:

In [32]:
#Plotting
xr = (8400,8900)
yr = (0.7,1.1)

#Not sure if I need to do this
if os.path.exists(sourcename+'cont.dispcor.fits'):
    os.remove(sourcename+'cont.dispcor.fits')

iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

x=[]
y=[]
with open(sourcename+'cont.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))

x = np.array(x)
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)


cx719ssemcont.fits: Resampling using current coordinate system
cx719ssemcont.dispcor.fits: ap = 1, w1 =   3501.3, w2 =   9996.1, dw =      2.6, nw = 2499


GlyphRenderer(id='b3424366-540a-478e-9c34-07f065a92e19', ...)

## Overplot lines

If desired we can define several know emission and absoprtion lines to overplot to the normalize spectra

In [35]:
##Emission Lines
class line(object):
    def __init__(self,name):
        self.name = name

diclines = {line('Ca II'):8498,line('Ca II'):8662,line('Ca II'):8541,
            line('O I'):8446,line('H'+u"\u03B1"):6563,line("(BaII, FeI and CaI)"):6497,
            line('O I Triplet(72,74,75)'):7774,line('He I'):7065,line('Fe I'):8621,
            line('P16'):8502,line('P15'):8545,line('P14'):8598,line('P13'):8665,
            line('P12'):8750,line('P11'):8863,line('P10'):9015,line('P9'):9229,
            line('P8'):9546,line('P17'):8467,line('P18'):8438,line('P19'):8413,line('P20'):8392}
  

for name, xloc in diclines.iteritems():
    yloc = y[np.where(abs(xloc - x) < 2)[0][0]]
    span = Arrow(end=NormalHead(fill_color='orange', size=10),
             x_start=xloc, y_start = yloc - .15, x_end = xloc, y_end= yloc-.03
            )
    plot.add_layout(span)
    my_label = Label(x=xloc, y=yloc-.03, text=name.name)
    plot.add_layout(my_label)

In [36]:
show(plot)

# The spectrum


Absoprtion H$\alpha$ and Paschen 

# Second Source

In [13]:
#----------#
#Variables-#
#----------#
sourcename = 
center = 
low = -5
high = 5
b_low = [0,0]
b_up = [40-center,67-center]
databasegeneral = 'database/'
filename = 'database/ap'+sourcename
fitsfilename = 
#----------#
#Exptime---#
#----------#
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    
iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)
#----------#
#Database--#
#----------#
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagenameor = lines
                imagename= lines.replace(lines.split()[1],sourcename)
        if 'center' in lines:
                numerocenteror = lines
                numerocenter = lines.replace(lines.split()[2], str(center))
        if 'low' in lines:
                numerolowor = lines
                numerolow = lines.replace(lines.split()[2],str(low))
        if 'high' in lines:
                numerohighor = lines
                numerohigh = lines.replace(lines.split()[2], str(high))
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],str(min(b_low)))
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],str(max(b_up)))
        if 'sample' in lines:
            sampleor = lines
            sampleb = '\t\tsample '+str(min(b_low))+':'+str(max(b_low))+','+str(min(b_up))+':'+str(max(b_up))+'\n'
            break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagenameor,imagename)
filedata = filedata.replace(numerocenteror,numerocenter)
filedata = filedata.replace(numerolowor,numerolow)
filedata = filedata.replace(numerohighor,numerohigh)
filedata = filedata.replace(bloworiginal,blow)
filedata = filedata.replace(buporiginal,bup)
filedata = filedata.replace(sampleor,sampleb)

                                
with open(filename,'w') as f:
    f.write(filedata)

#----------#
#.par------#
#----------#
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')
#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','average')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

SyntaxError: invalid syntax (<ipython-input-13-794f6322e47d>, line 4)

In [10]:
%%bash -s "$filename"
cat $1

cat: $filename: No such file or directory


## Plots

In [14]:
#Not sure if I need to do this
if os.path.exists(sourcename+'.dispcor.fits'):
    os.remove(sourcename+'.dispcor.fits')

iraf.dispcor(sourcename+'.ms.fits',sourcename+'.dispcor.fits')
iraf.wspectext(sourcename+'.dispcor.fits[*,1,1]',sourcename+'.txt',header='no')

x=[]
y=[]
with open(sourcename+'.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))


source = ColumnDataSource(data=dict(x=x,y=y))
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra",
              active_scroll='wheel_zoom',plot_width=900, plot_height=700)
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
show(plot)

AttributeError: Undefined IRAF task `dispcor'

## Normalize


In [15]:
sample = '5000:9000'
if os.path.exists(sourcename+'cont.fits'):
    os.remove(sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('input',sourcename+'.ms.fits')
iraf.noao.onedspec.continuum.setParam('output',sourcename+'cont.fits')
iraf.noao.onedspec.continuum.setParam('interactive','no')
iraf.noao.onedspec.continuum.setParam('sample',sample)
iraf.noao.onedspec.continuum.saveParList(filename='uparm/cont'+sourcename+'.par')
iraf.noao.onedspec.continuum(ParList='uparm/cont'+sourcename+'.par')

#Plotting
xr = (8400,8900)
yr = (0.7,1.1)

#Not sure if I need to do this
if os.path.exists(sourcename+'cont.dispcor.fits'):
    os.remove(sourcename+'cont.dispcor.fits')

iraf.dispcor(sourcename+'cont.fits',sourcename+'cont.dispcor.fits')
iraf.wspectext(sourcename+'cont.dispcor.fits[*,1,1]',sourcename+'cont.txt',header='no')

x=[]
y=[]
with open(sourcename+'cont.txt') as f:
    for lines in f:
        x.append(float(lines.split()[0]))
        y.append(float(lines.split()[1]))

x = np.array(x)
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1.11}, $y)"),
        ]
    )

source = ColumnDataSource(data=dict(x=x,y=y))

plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=700
             )
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
plot.line('x','y',source=source)
##Emission Lines
class line(object):
    def __init__(self,name):
        self.name = name


diclines = {line('Ca II'):8498,line('Ca II'):8662,line('Ca II'):8541,line('Fe I'):8621,line('Fe I'):8688,
            line('O I'):8446,line('Fe I'):8514,line('Fe I'):8468,
           line('H'+u"\u03B1"):6563,line("(BaII, FeI and CaI)"):6497,
            line('Ca I'):6162,line('Mg I'):8807}

for name, xloc in diclines.iteritems():
    yloc = y[np.where(abs(xloc - x) < 2)[0][0]]
    span = Arrow(end=NormalHead(fill_color='orange', size=10),
             x_start=xloc, y_start = yloc - .15, x_end = xloc, y_end= yloc-.03
            )
    plot.add_layout(span)
    my_label = Label(x=xloc, y=yloc-.03, text=name.name)
    plot.add_layout(my_label)
show(plot)

AttributeError: Package onedspec has not been loaded; no tasks are defined

# References